In [1]:
import sys; sys.path.append("../../automl/")

In [2]:
from pathlib import Path
import yaml
import joblib

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from src.automl.model.xgboost import XGBClassification
from src.automl.model.lama import TabularLama
from src.automl.model.lightgbm import LightGBMClassification

from src.automl.loggers import configure_root_logger
from src.automl.constants import create_ml_data_dir
from src.automl.model.metrics import RocAuc

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
[2024-11-06 12:51:42]
/home/peter/venvs/base_venv/lib/python3.10/site-packages/lightautoml/ml_algo/dl_model.py:42: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/

## Constants

In [4]:
RANDOM_SEED = 77
DATA_PATH = Path("../../data/")
CONFIG_PATH = Path("../../configs/config.yaml")
N_JOBS = 16

In [5]:
with CONFIG_PATH.open() as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)

## Data

In [6]:
df_train = pd.read_parquet(DATA_PATH / "train_preproc.parquet")
df_train, df_test = df_train.sort_values(by="id").iloc[:300_000], df_train.sort_values(by="id").iloc[300_000:]

In [7]:
df_train["target"].value_counts()

0    282256
1     17744
Name: target, dtype: int64

In [8]:
# undersample the 0 class
df_train = pd.concat([df_train.loc[df_train.target == 1], df_train.loc[df_train.target == 0].sample(200_000, random_state=RANDOM_SEED)], ignore_index=True)

In [9]:
# add oof columns
oof_paths = ["../../data/models/lama_8029/oof.csv",
             "../../data/models/lgb_8055/oof.csv",
             "../../data/models/xgb_8052/oof.csv"]
df_oof = pd.read_csv(oof_paths[0])
for oof_file in oof_paths[1:]:
    temp = pd.read_csv(oof_file)
    df_oof = pd.concat([df_oof, temp], axis=1)

In [10]:
df_oof["target"] = df_train["target"]

In [11]:
df_train = pd.concat([df_train.drop(columns="target"), df_oof], axis=1)

In [12]:
ohe_cols = df_train.columns[df_train.columns.str.startswith("OneHotEncoder")].values.tolist()
oe_cols = df_train.columns[df_train.columns.str.startswith("OrdinalEncoder")].values.tolist()
te_cols = df_train.columns[df_train.columns.str.startswith("MeanTargetEncoder")].values.tolist()

In [17]:
# take target encoded columns
X_train, y_train = df_train[cfg["selected_features"] + te_cols + ohe_cols + list(df_oof.columns[: -1])], df_train["target"]
#X_test, y_test = df_test[cfg["selected_features"] + te_cols + ohe_cols + list(df_oof.columns[: -1])], df_test["target"]

In [13]:
# take all columns
X_train, y_train = df_train.drop(columns=["target", "id", "smpl"]), df_train["target"]
X_test, y_test = df_test.drop(columns=["target", "id", "smpl"]), df_test["target"]

In [73]:
# take ordinal encoded columns
X_train, y_train = df_train[cfg["selected_features"] + oe_cols + ohe_cols], df_train["target"]
X_test, y_test = df_test[cfg["selected_features"] + oe_cols + ohe_cols], df_test["target"]

In [13]:
display(y_train.value_counts(normalize=True))
display(y_test.value_counts(normalize=True))

0    0.91851
1    0.08149
Name: target, dtype: float64

0    0.941322
1    0.058678
Name: target, dtype: float64

In [18]:
categorical_features = ohe_cols# + oe_cols
test = pd.read_parquet(DATA_PATH / "test_preproc.parquet")

## Blend

In [41]:
from sklearn.model_selection import StratifiedKFold
from tqdm.auto import tqdm

best_score = 0
best_pred = None
best_weights = (None, None, None)
metric = RocAuc()

#kf = StratifiedKFold(n_splits=5, random_state=self.random_state, shuffle=True)
for weights_1 in tqdm(np.arange(0, 1, 0.005)):
    for weights_2 in np.arange(0, 1 - weights_1, 0.005):
        weights_3 = 1 - weights_2 - weights_1

        pred = (weights_1 * df_oof.iloc[:, 0].values +
                weights_2 * df_oof.iloc[:, 1].values +
                weights_3 * df_oof.iloc[:, 2].values)

        score = metric(df_oof["target"], pred)
        
        if score > best_score:
            best_score = score
            best_pred = pred
            best_weights = (weights_1, weights_2, weights_3)

print(best_score)
print(best_weights)
print()

  0%|          | 0/200 [00:00<?, ?it/s]

[2024-11-06 12:07:35]

KeyboardInterrupt



In [39]:
model_1 = joblib.load("../../data/models/lama_8029/lama_8029.joblib")
model_1.task = "binary"

model_2 = joblib.load("../../data/models/lgb_8055/lgb_8055.joblib")
model_3 = joblib.load("../../data/models/xgb_8052/xgb_8052.joblib")

pred_1 = model_1.predict(test[cfg["selected_features"] + te_cols + ohe_cols])
pred_2 = model_2.predict(test[cfg["selected_features"] + te_cols + ohe_cols])
pred_3 = model_3.predict(test[cfg["selected_features"] + te_cols + ohe_cols])

In [42]:
test["target"] = pred_1[:, 1] * 0.12 + pred_2[:, 1] * 0.52 + pred_3[:, 1] * 0.36

In [45]:
test[['id', 'target']].to_csv('blend_806.csv', index=False)

In [32]:
print(np.array(cfg["selected_features"] + te_cols + ohe_cols)[np.argsort(-model_3.models[0].feature_importances_)])
print(model_3.models[0].feature_importances_[np.argsort(-model_3.models[0].feature_importances_)])

['numeric__feature_16' 'numeric__feature_11' 'numeric__feature_100'
 'numeric__feature_142' 'numeric__feature_93' 'numeric__feature_8'
 'numeric__feature_46' 'numeric__feature_168' 'numeric__feature_95'
 'OneHotEncoder__feature_71_2' 'numeric__feature_141'
 'numeric__feature_183' 'numeric__feature_33' 'numeric__feature_62'
 'numeric__feature_25' 'numeric__feature_18' 'numeric__feature_6'
 'numeric__feature_26' 'numeric__feature_43' 'numeric__feature_59'
 'numeric__feature_12' 'numeric__feature_78'
 'MeanTargetEncoder__feature_179' 'numeric__feature_20'
 'numeric__feature_157' 'OneHotEncoder__feature_31_3'
 'numeric__feature_90' 'numeric__feature_91' 'numeric__feature_79'
 'numeric__feature_76' 'numeric__feature_10' 'numeric__feature_50'
 'numeric__feature_44' 'numeric__feature_145' 'numeric__feature_112'
 'numeric__feature_111' 'OneHotEncoder__feature_31_2'
 'numeric__feature_96' 'MeanTargetEncoder__feature_163'
 'numeric__feature_94' 'numeric__feature_36'
 'MeanTargetEncoder__feature_

## Stack

In [20]:
metric = RocAuc()

In [22]:
model = TabularLama(n_jobs=N_JOBS, task="classification")
model.tune(X_train, y_train, metric, timeout=60 * 60)
oof = model.fit(X_train, y_train)
#y_pred = model.predict(X_test)

print(metric(y_train, oof))

[2024-11-06 12:55:19,579] - [   START    ] - Fitting TabularLama
[12:55:19] Stdout logging level is INFO.
[12:55:19] Task: binary

[12:55:19] Start automl preset with listed constraints:
[12:55:19] - time: 7200.00 seconds
[12:55:19] - CPU: 16 cores
[12:55:19] - memory: 16 GB

[12:55:19] Train data shape: (217744, 71)

[12:55:31] Layer 1 train process start. Time left 7187.74 secs
[12:55:32] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[12:55:35] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.8007066986023443
[12:55:35] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[12:55:35] Time left 7184.21 secs

[12:55:38] Selector_LightGBM fitting and predicting completed
[12:55:38] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[12:55:52] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = 0.7919720626690712
[12:55:52] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predicting completed
[12:55:52] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ... Time b

## Model

In [16]:
metric = RocAuc()

In [17]:
model = XGBClassification(n_jobs=16)
model.tune(X_train, y_train, metric, timeout=60 * 60, categorical_features=categorical_features)
oof = model.fit(X_train, y_train, categorical_features=categorical_features)
y_pred = model.predict(X_test)

print(metric(y_train, oof))
print(metric(y_test, y_pred))

[2024-11-06 09:19:23,754] - [   START    ] - Tuning XGBClassification
[2024-11-06 09:20:04,920] - [   OPTUNA   ] - Trial 0. New best score 0.8004664892928378 with parameters {'max_depth': 6, 'grow_policy': 'depthwise', 'max_leaves': 311, 'gamma': 3.1203728088487304, 'subsample': 0.2403950683025824, 'colsample_bytree': 0.15227525095137953, 'colsample_bylevel': 0.8795585311974417, 'reg_lambda': 6.011150117432088, 'reg_alpha': 7.080725777960454, 'min_child_weight': 0, 'class_weight': None, 'n_estimators': 819}
[2024-11-06 09:21:23,947] - [   OPTUNA   ] - Trial 1. New best score 0.8012945751647573 with parameters {'max_depth': 4, 'grow_policy': 'lossguide', 'max_leaves': 163, 'gamma': 10.495128632644757, 'subsample': 0.48875051677790415, 'colsample_bytree': 0.36210622617823773, 'colsample_bylevel': 0.6506676052501416, 'reg_lambda': 1.3949386065204183, 'reg_alpha': 2.9214464853521815, 'min_child_weight': 7, 'class_weight': 'balanced', 'n_estimators': 1998}
[2024-11-06 09:31:02,977] - [   OP

In [18]:
MODEL_NAME = "xgb_8052"
MODEL_DIR = Path(f"../../data/models/{MODEL_NAME}")
MODEL_DIR.mkdir(exist_ok=True)

In [19]:
res = pd.DataFrame()
res[MODEL_NAME] = oof[:, 1]
res.to_csv(MODEL_DIR / "oof.csv", index=False)
joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

with (MODEL_DIR / "params.yaml").open("w") as f:
    yaml.dump(model.params, f)

with (MODEL_DIR / "score.txt").open("w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    print("Test:", metric(y_test, y_pred), file=f)

In [154]:
joblib.dump(model, "../../data/models/lgb_8055/lgb_8055.joblib")


['../../data/models/lgb_8055/lgb_8055.joblib']

In [158]:
with open("../../data/models/lgb_8055/params.yaml", "w") as f:
    yaml.dump(model.params, f)

In [161]:
with open("../../data/models/lgb_8055/score.txt", "w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    print("Test:", metric(y_test, y_pred), file=f)

In [145]:
print(metric(y_train, oof))
print(metric(y_test, y_pred))

0.8047647204688908
0.7972289963856585


In [86]:
print(metric(y_train, oof))
print(metric(y_test, y_pred))

0.8009657912533814
0.7932043263129364


In [47]:
print(metric(y_train, oof))
print(metric(y_test, y_pred))

0.8036673066952209
0.7969396115051322


In [37]:
print(metric(y_train, oof))
print(metric(y_test, y_pred))

0.8042716798166517
0.7975841531905153


In [15]:
test = pd.read_parquet(DATA_PATH / "test_preproc.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + te_cols + ohe_cols])[:, 1]
test[['id', 'target']].to_csv('lgb_full_dataset.csv', index=False)

In [25]:
pred_1 = pd.read_csv("lama_utilized.csv")
pred_2 = pd.read_csv("lgmb_oe_ohe_cols_0805.csv")
pred_3 = pd.read_csv("catboost_ts.csv")

In [26]:
pred_1["target"] = 0.6 * pred_1["target"] + 0.2 * pred_2["target"] + 0.2 * pred_3["target"]

In [29]:
pred_1.to_csv("blend.csv", index=False)

In [166]:
MODEL_DIR.open?

Signature:
MODEL_DIR.open(
    mode='r',
    buffering=-1,
    encoding=None,
    errors=None,
    newline=None,
)
Docstring:
Open the file pointed by this path and return a file object, as
the built-in open() function does.
File:      /usr/lib/python3.10/pathlib.py
Type:      method